# Lynbrook ML Club
## 1/14/2022
## CNN with Pytorch

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kentonishi/ml-club-mnist-pytorch/blob/master/main.ipynb)

In [ ]:
# install pytorch and torchvision
%pip install torch torchvision

In [ ]:
# import torch and torchvision 
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# create datasets and dataloaders
to_tensor = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=to_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=to_tensor)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
# define the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1)
        self.conv1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = torch.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = torch.log_softmax(x, dim=1)
        return output

In [ ]:
# instantiate the model, loss function, and optimizer
current_epoch = 0
num_epochs = 5
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Net().to(device)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# define train and test functions

def train_iteration():
    model.train()
    for batch_idx, (images, labels) in enumerate(train_dataloader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0 or batch_idx == len(train_dataloader) - 1:
            print(
                f"Epoch: {current_epoch}/{num_epochs},",
                f"Batch {batch_idx}/{len(train_dataloader)},",
                f"Loss: {loss.item()}", end="\r")
    print()


def test():
    model.eval()
    correct_count = 0
    total_count = 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            _, predicted = torch.max(output.data, 1)
            total_count += labels.size(0)
            correct_count += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct_count / total_count}%")

def save():
    torch.save(model.state_dict(), f"model-{current_epoch}.pt")
    torch.save(optimizer.state_dict(), f"optimizer-{current_epoch}.pt")

In [ ]:
# training loop
while current_epoch < num_epochs:
    train_iteration()
    test()
    save()
    current_epoch += 1